In [1]:
import torch
from functions import CustomSampler, load_pretrained_mlp, load_pretrained_cae, get_model_predictions, average_fusion_predictions, calculate_classification_report, calculate_fusion_accuracy
from ae_functions import get_latent_dataloader
from ae_models import DeepCAE, DeepBatchCAE, SimpleCAE, SimpleBatchCAE, SimpleCAE_Dropout, SimpleBatchDropoutCAE
from mlp_functions import train_mlp, validate_mlp, decision_level_fusion
from mlp_models import rawMLP, MLP, dropout_MLP, ImprovedMLP
import matplotlib.pyplot as plt

In [2]:
##### CONFIG
output_dim = 4
input_dim = 147456
BATCH_SIZE = 8

DEVICE = ""
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("CUDA is available. Using GPU...")
else:
    DEVICE = torch.device("cpu")
    print("CUDA is not available. Using CPU...")

# Define the tool names and actions
TOOL_NAMES = ['hook', 'ruler', 'spatula', 'sshot']
ACTIONS = ['left_to_right', 'pull', 'push', 'right_to_left']

# All available object names
OBJECTS = ['0_woodenCube', '1_pearToy', '2_yogurtYellowbottle', '3_cowToy', '4_tennisBallYellowGreen',
            '5_blackCoinbag', '6_lemonSodaCan', '7_peperoneGreenToy', '8_boxEgg','9_pumpkinToy',
            '10_tomatoCan', '11_boxMilk', '12_containerNuts', '13_cornCob', '14_yellowFruitToy',
            '15_bottleNailPolisher', '16_boxRealSense', '17_clampOrange', '18_greenRectangleToy', '19_ketchupToy']

BASE_PATH = 'C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/fusion'

sensor_color = "color"
sensor_left = "icub_left"
sensor_right = "icub_right"
sensor_depth = "depthcolormap"

CUDA is available. Using GPU...


In [11]:
# Load models
model_path = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/weights_ae/"
cae_color = load_pretrained_cae(SimpleBatchCAE, model_path, "batchnorm/batch_cae_ne5_b8_color.pth", DEVICE)
cae_left = load_pretrained_cae(SimpleBatchCAE, model_path, "batchnorm/batch_cae_ne5_b8_left.pth", DEVICE)
cae_right = load_pretrained_cae(SimpleBatchCAE, model_path, "batchnorm/batch_cae_ne5_b8_right.pth", DEVICE)

# Load MLP models
model_path = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/weights_mlp/"
mlp_color = load_pretrained_mlp(MLP, model_path, "batchnorm/mlp_ne5_b8_color.pth", DEVICE, input_dim, output_dim)
mlp_left = load_pretrained_mlp(MLP, model_path, "batchnorm/mlp_ne5_b8_left.pth", DEVICE, input_dim, output_dim)
mlp_right = load_pretrained_mlp(MLP, model_path, "batchnorm/mlp_ne5_b8_right.pth", DEVICE, input_dim, output_dim)

models = [mlp_color, mlp_left, mlp_right]

# Assuming the datasets for all sensors are the same size
dataset_size = 640  # Replace with the actual size of your dataset
shuffled_indices = torch.randperm(dataset_size).tolist()

# Get dataloaders
set_name = "validation"
test_loader_color = get_latent_dataloader(cae_color, BASE_PATH, OBJECTS, TOOL_NAMES, ACTIONS, sensor_color, set_name, shuffle=False, batch_size=BATCH_SIZE, device=DEVICE, indices=shuffled_indices)
test_loader_left = get_latent_dataloader(cae_left, BASE_PATH, OBJECTS, TOOL_NAMES, ACTIONS, sensor_left, set_name, shuffle=False, batch_size=BATCH_SIZE, device=DEVICE, indices=shuffled_indices)
test_loader_right = get_latent_dataloader(cae_right, BASE_PATH, OBJECTS, TOOL_NAMES, ACTIONS, sensor_right, set_name, shuffle=False, batch_size=BATCH_SIZE, device=DEVICE, indices=shuffled_indices)

dataloaders = [test_loader_color, test_loader_left, test_loader_right]

# Get predictions
model_predictions = [get_model_predictions(model, dataloader, DEVICE) for model, dataloader in zip(models, dataloaders)]

# Combine and average predictions
final_predictions = average_fusion_predictions(model_predictions)

# Convert to predicted classes
predicted_classes = [pred.argmax(dim=-1) for pred in final_predictions]
predicted_classes_list = [pred.tolist() if torch.is_tensor(pred) else pred for pred in predicted_classes]

# Gather true labels and calculate accuracy
class_names = ['left_to_right', 'pull', 'push', 'right_to_left']
true_labels = [labels.tolist() for _, labels in test_loader_color]
accuracy = calculate_fusion_accuracy(predicted_classes_list, true_labels)
calculate_classification_report(predicted_classes_list, true_labels, class_names)
print(accuracy)

               precision    recall  f1-score   support

left_to_right       0.91      0.98      0.95       160
         pull       0.99      0.89      0.94       160
         push       0.94      0.96      0.95       160
right_to_left       0.97      0.97      0.97       160

     accuracy                           0.95       640
    macro avg       0.95      0.95      0.95       640
 weighted avg       0.95      0.95      0.95       640

0.953125


In [3]:
# # Load CAE models
# model_path = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/weights_ae/"

# weight_name = "batchnorm/batch_cae_ne5_b8_color.pth"
# cae_color = SimpleBatchCAE().to(DEVICE)
# cae_color.load_state_dict(torch.load(model_path+weight_name))

# weight_name = "batchnorm/batch_cae_ne5_b8_left.pth"
# cae_left = SimpleBatchCAE().to(DEVICE)
# cae_left.load_state_dict(torch.load(model_path+weight_name))

# weight_name = "batchnorm/batch_cae_ne5_b8_right.pth"
# cae_right = SimpleBatchCAE().to(DEVICE)
# cae_right.load_state_dict(torch.load(model_path+weight_name))

# # Load MLP models
# model_path = "C:/Users/Frank/OneDrive/Bureaublad/ARC/deep-multimodal-learning/weights_mlp/"

# weight_name = "batchnorm/mlp_ne5_b8_color.pth"
# mlp_color = MLP(input_dim, output_dim).to(DEVICE)  # Replace 'input_dim' and 'output_dim' with appropriate values
# mlp_color.load_state_dict(torch.load(model_path+weight_name))
# mlp_color.eval()

# weight_name = "batchnorm/mlp_ne5_b8_left.pth"
# mlp_left = MLP(input_dim, output_dim).to(DEVICE)  # Replace 'input_dim' and 'output_dim' with appropriate values
# mlp_left.load_state_dict(torch.load(model_path+weight_name))
# mlp_left.eval()

# weight_name = "batchnorm/mlp_ne5_b8_right.pth"
# mlp_right = MLP(input_dim, output_dim).to(DEVICE)  # Replace 'input_dim' and 'output_dim' with appropriate values
# mlp_right.load_state_dict(torch.load(model_path+weight_name))
# mlp_right.eval()

# models = [mlp_color, mlp_left, mlp_right]

# # Get data loaders -> latent dataset loaders
# test_loader_color = get_loader(BASE_PATH, OBJECTS, TOOL_NAMES, ACTIONS, sensor_color, "testing", batch_size=BATCH_SIZE)
# test_loader_right = get_loader(BASE_PATH, OBJECTS, TOOL_NAMES, ACTIONS, sensor_right, "testing", batch_size=BATCH_SIZE)
# test_loader_left = get_loader(BASE_PATH, OBJECTS, TOOL_NAMES, ACTIONS, sensor_left, "testing", batch_size=BATCH_SIZE)

# dataset_color = get_latent_dataset(cae_color, test_loader_color, add_noise=False, is_depth=False, device=DEVICE)
# test_loader_color = torch.utils.data.DataLoader(dataset_color, batch_size=BATCH_SIZE, shuffle=False)

# dataset_left = get_latent_dataset(cae_left, test_loader_left, add_noise=False, is_depth=False, device=DEVICE)
# test_loader_left = torch.utils.data.DataLoader(dataset_left, batch_size=BATCH_SIZE, shuffle=False)

# dataset_right = get_latent_dataset(cae_right, test_loader_right, add_noise=False, is_depth=False, device=DEVICE)
# test_loader_right = torch.utils.data.DataLoader(dataset_right, batch_size=BATCH_SIZE, shuffle=False)

# dataloaders = [test_loader_color, test_loader_left, test_loader_right]


# model_predictions = []
# # Assuming 'models' is a list of your trained models and 'dataloaders' is the corresponding list of dataloaders
# for model, dataloader in zip(models, dataloaders):
#     predictions = []

#     with torch.no_grad():  # No need to calculate gradients
#         for inputs, _ in dataloader:  # Labels are not needed for prediction
#             inputs = inputs.to(DEVICE)  # Move the input data to the GPU
#             outputs = model(inputs)
#             # Convert outputs to desired format (e.g., probabilities, class indices)
#             predictions.append(outputs)
    
#     model_predictions.append(predictions)

# # Example for averaging probabilities (assuming softmax outputs)
# final_predictions = []
# for i in range(len(dataloaders[0])):  # Assuming all dataloaders have the same length
#     # Average predictions across models for each sample
#     avg_prediction = sum(model_predictions[j][i] for j in range(len(models))) / len(models)
#     final_predictions.append(avg_prediction)
    
# predicted_classes = [pred.argmax(dim=-1) for pred in final_predictions]

# true_labels = {0:[],1:[],2:[]}
# for i, dataloader in enumerate(dataloaders):
#     for _, labels in dataloader:
#         true_labels[i].append(labels.tolist())

# # Convert predicted classes to lists if they are tensors
# predicted_classes_list = [pred.tolist() if torch.is_tensor(pred) else pred for pred in predicted_classes]

# # Flatten the list of lists for true labels and predicted classes
# flattened_true_labels = [label for sublist in true_labels[0] for label in sublist]
# flattened_predicted_classes = [pred for sublist in predicted_classes_list for pred in sublist]

# # Ensure that the lengths of flattened true labels and predicted classes are the same
# assert len(flattened_true_labels) == len(flattened_predicted_classes), "Mismatch in the number of predictions and true labels"

# # Calculate the number of correct predictions
# correct_predictions = sum(pred == true for pred, true in zip(flattened_predicted_classes, flattened_true_labels))

# # Calculate total predictions and accuracy
# total_predictions = len(flattened_true_labels)
# accuracy = correct_predictions / total_predictions

# print(f"Accuracy: {accuracy:.4f}")

<All keys matched successfully>